In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request
path = "resultados/reativo/sync.csv"
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))
requestscopy = requests.copy()

Hosts inicializados
Canais inicializados
Pares EPRs adicionados


In [2]:
import csv
import os

# Função para registrar os dados no CSV
def registrar_request_no_csv(request, estado_registro, filename="requests_data.csv"):
    """
    Registra as informações da request no CSV.
    - request: objeto da request a ser registrada.
    - estado_registro: se a request foi nova ou já tinha registro.
    - filename: nome do arquivo CSV (padrão: "requests_data.csv").
    """
    # Verifica se o arquivo já existe para adicionar o cabeçalho se necessário
    file_exists = os.path.exists(filename)
    
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # Se o arquivo não existe, adiciona o cabeçalho
        if not file_exists:
            writer.writerow(["ID", "Alice", "Bob", "Fidelidade Mínima", "Número de EPRs", "Início", "Término", "Novo Registro"])
        
        # Escreve os dados da request
        writer.writerow([str(request), request.alice, request.bob, request.fmin, request.neprs, request.starttime, request.endtime, estado_registro])

# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    current_time = 0  # Tempo global da simulação
    for request in requests:
        print(f"[Time {current_time}]")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = current_time
            current_time += 3  # Delay de 3 time-slots para registrar nova regra
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = current_time
            
            # Registra no CSV como um novo registro
            registrar_request_no_csv(request, "Novo Registro", path)
        else:  # Caso já exista a regra
            request.starttime = current_time
            current_time += 1  # Delay de 1 time-slot para execução
            controlador.run_rule(rule[1])
            request.endtime = current_time
            
            # Registra no CSV como já registrado
            registrar_request_no_csv(request, "Já Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")


In [3]:
simulate_requests_with_timeslots(rede, requests, controlador)

[Time 0]
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(5-1-2)
Running Swap(5-1-2)
Request R-5142F0: Start Time = 0, End Time = 3
[Time 3]
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(4-5-6)
Running Swap(4-5-6)
Request R-00E490: Start Time = 3, End Time = 6
[Time 6]
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(7-6-4)
Running Swap(7-6-4)
Request R-00E5D0: Start Time = 6, End Time = 9
[Time 9]
Decisão aplicada: (Normal E2E,)
Passo: 1
Request R-006C40: Start Time = 9, End Time = 12
[Time 12]
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(3-2-1)
Running Swap(3-2-1)
Request R-006D70: Start Time = 12, End Time = 15
[Time 15]
Decisão aplicada: (High Fidelity,)
Request R-FBB0B0: Start Time = 15, End Time = 18
[Time 18]
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
    Ação: Swap(8-4-0)
Running Swap(8-4-0)
Request R-0668B0: Start Time = 18, End Time = 21
[Time 21]
Decisão aplicada: (Normal E2E,)
Passo: 1
Passo: 2
  